<img src='https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iYYkEpi_S2Bk/v1/1000x-1.jpg' style='height:400px'>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div class="list-group" id="list-tab" role="tablist">
  <h1 class="list-group-item list-group-item-action active" data-toggle="list"  role="tab" aria-controls="home">&nbsp;Summary:</h1>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab" aria-controls="profile">1. Introduction<span class="badge badge-primary badge-pill">1</span></a>
  <a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab" aria-controls="messages">2. Time Series<span class="badge badge-primary badge-pill">2</span></a>
</div>

<a id="1"></a> <br>
<font size="+3" color="black"><b>1 - Introduction</b></font><br><a id="1"></a>
<br> 
* the propose of this kernel is a detailed view in temporary series for fires in brazil, exploring some components of the series

In [ ]:
df = pd.read_csv('/kaggle/input/forest-fires-in-brazil/amazon.csv', encoding = "ISO-8859-1")
df.head()

* we will define our time intervals in quarters to analyze the fires

<a id="1"></a> <br>
<font size="+3" color="black"><b>2 - Time Series Analysis</b></font><br><a id="2"></a>
<br> 

* Lets start exploring our time series

In [ ]:
list_ = range(1, 13)
m = dict(zip(df['month'].unique(), list_))
df['month'] = df['month'].map(m)

In [ ]:
def quarter(month):
    if month == 1 or month == 2 or month == 3:
        return '1_Quarter'
    elif month == 4 or month == 5 or month == 6:
        return '2_Quarter'
    elif month == 7 or month == 8 or month == 9:
        return '3_Quarter'
    elif month == 10 or month == 11 or month == 12:
        return '4_Quarter'
    
    
df['quarter'] = df['month'].apply(lambda x : quarter(x))

In [ ]:
aux = df.groupby(['year', 'quarter'])['number'].agg('sum')
indexs = []
for year,quarter in list(aux.index):
    indexs.append(str(year)+'_'+str(quarter))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.line(x=indexs, y=list(aux[:]), title='Observed')
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

* The series show that there are some seasonality the first and secound quarter in each year is the period where there are less fires, however the third and the fourth are the peak burn rates, so to identify if it is behavior of series i will observer if this series above is a stationary series, if is a stationary means that this flow of fires is natural occur at this quarter division

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

adf_test(list(aux[:]))

* the p-value is greater than 0 so our series is not stationary, but the value is so small if is interisting we can consider, stationary series has some interesting properties like mean, variacian and covariancia being constant throughout the series

* so now lets we will decompose the series into the components that compose it

In [ ]:
import statsmodels.api as sm
import plotly.graph_objects as go

res = sm.tsa.seasonal_decompose(list(aux[:]), period=3, model="additive")
fig = go.Figure()

fig.add_trace(go.Scatter(x=indexs, y=res.observed, mode='lines',name='Obeserved'))
fig.add_trace(go.Scatter(x=indexs, y=res.trend, mode='lines',name='Trend'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
fig = px.line(x=indexs, y=res.resid, title='Residual component')
fig.show()

* So now we know about component series, lets know how each state contributes to series constructor

In [ ]:
def get_correlation_between_states(df):
    dict_ = {}
    for state in df['state'].unique():
        select = df[df['state']==state]
        aux2 = select.groupby(['year', 'quarter'])['number'].agg('sum')
        aux = df.groupby(['year', 'quarter'])['number'].agg('sum')
        #indexs = []
        #for year,quarter in list(aux.index):
        #    indexs.append(str(year)+'_'+str(quarter))
        dict_[state] =  np.corrcoef(aux2[:], aux[:])[0][1]
    return dict_

dict_ =  get_correlation_between_states(df)

* In the data there are 23 brazilian states, to better undestand the series lets see how burned in each state are correlated with the long serie, to do this i will use pearson correlation 

In [ ]:
import plotly.express as px

sorted_dict = sorted(dict_.items(), key=lambda kv: kv[1])
sorted_dict = dict(sorted_dict)
fig = px.bar(x=list(sorted_dict.keys()), y=list(sorted_dict.values()))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
def get_state(df, state):
    aux = df[df['state']==state]
    aux2 = aux.groupby(['year', 'quarter'])['number'].agg('sum')
    indexs = []
    for year, quarter in list(aux2.index):
        indexs.append(str(year)+'_'+str(quarter))
    return indexs, list(aux2[:])

* in the graph above we can see the states that most correlate with the general graph of fires, this shows that these states follow the growth or decline of fires in the country, however we can have a biased analysis in this regard to better understand our series we will remove the trend component of the series and see how the correlations remain

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(indexs), y=list(aux[:]), mode='lines',name='Observed'))
x, y = get_state(df, 'Sergipe')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Sergipe'))
x, y = get_state(df, 'Roraima')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Roraima'))
x, y = get_state(df, 'Maranhao')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Maranhao'))
x, y = get_state(df, 'Rio')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Rio'))
x, y = get_state(df, 'Acre')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Acre'))
x, y = get_state(df, 'Paraiba')
fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Paraiba'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff

def get_correlation_between_states_detrend(df):
    dict_ = {}
    for state in df['state'].unique():
        select = df[df['state']==state]
        aux2 = select.groupby(['year'])['number'].agg('sum')
        aux = df.groupby(['year'])['number'].agg('sum')
        detrend2 = difference(list(aux2[:]))
        detrend = difference(list(aux[:]))
        dict_[state] =  np.corrcoef(detrend2, detrend)[0][1]
    return dict_

In [ ]:
dict_ = get_correlation_between_states_detrend(df)
sorted_dict = sorted(dict_.items(), key=lambda kv: kv[1])
sorted_dict = dict(sorted_dict)
fig = px.bar(x=list(sorted_dict.keys()), y=list(sorted_dict.values()))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

* after removing the trend component we have changes in the correlations, let us show in fact how each state correlates with the fires in the country over the years


In [ ]:
fig = go.Figure()
y = difference(list(aux[:]))

fig.add_trace(go.Scatter(x=indexs, y=y, mode='lines',name='Observed'))

x, y = get_state(df, 'Distrito Federal')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Distrito Federal'))
x, y = get_state(df, 'Rondonia')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Rondonia'))

x, y = get_state(df, 'Amazonas')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Amazonas'))

x, y = get_state(df, 'Pernambuco')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Pernambuco'))
x, y = get_state(df, 'Bahia')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Bahia'))
x, y = get_state(df, 'Mato Grosso')
y = difference(y)

fig.add_trace(go.Scatter(x=x, y=y, mode='lines',name='Mato Grosso'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
def fires_count(df):
    dict_ ={}
    for state in df['state'].unique():
        aux = df[df['state']==state]
        tmp = aux.groupby(['state'])['number'].agg('sum')
        dict_[state] = sum(tmp[:])
    return dict_


dict_ = fires_count(df)
sorted_dict = sorted(dict_.items(), key=lambda kv: kv[1])
sorted_dict = dict(sorted_dict)
fig = px.bar(x=list(sorted_dict.keys()), y=list(sorted_dict.values()))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

In [ ]:
fig = px.line(x=indexs, y=res.seasonal, title='Residual component')
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()

 * above we can observe that the number of fires by ranked states, however observing the seasonality component of the series we see that it is periodic, so there are states that are more susceptible to fires depending on the time of year, and then we will observe how are the states with more fires if we disregard this component


In [ ]:
def fires_count_deseasonality(df):    
    for state in df['state'].unique():
        aux = df[df['state']==state]
        tmp = aux.groupby(['year', 'quarter'])['number'].agg('sum')
        y = difference(list(tmp[:]), 3)
        dict_[state] = sum(y)
    return dict_

dict_ = fires_count_deseasonality(df)
sorted_dict = sorted(dict_.items(), key=lambda kv: kv[1])
sorted_dict = dict(sorted_dict)
fig = px.bar(x=list(sorted_dict.keys()), y=list(sorted_dict.values()))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.show()